In [54]:
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
import rdata
import numpy as np

In [55]:
nacc_test = pd.read_csv(r'C:\Users\jaiga\Downloads\UDS3Mod_for_Jai_Crosswalk_Levels_only.csv')
nacc_test.head(3)

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_data_access_group,redcap_survey_identifier,adc_sub_id,visitmo,visitday,visityr,...,pbmc_count,pbmc_dt,plasma_count,plasma_dt,serum_count,serum_dt,urine_count,urine_dt,biospe_dde_notes,biospecimen_inventory_complete
0,104,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1105,1,14,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1102,1,13,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,106,visit_info_arm_1,NaN,1,crosswalk_data,NaN,ADC1129,2,5,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
nacc_test.shape

(3, 2900)

In [57]:
#nacc_test = nacc_test.head(3)

In [58]:
# Load JSON mapping file
with open(r'C:\Users\jaiga\Downloads\crosswalks\A1_UDS4_programmatic_crosswalk_mappings.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

In [59]:
mapping_data.keys()

dict_keys(['Direct_Mappings', 'Conditional_Consistency', 'Structured_Transformations', 'High_Complexity'])

In [60]:
uds3_df = nacc_test
uds3_df.columns = uds3_df.columns.str.lower()
uds4_df = pd.DataFrame()

data_dict = {'ptid__a1': 'adc_sub_id', 'visitnum__a1': 'a1_visit_day','initials__a1':'a1_ex_ini'}

In [61]:
uds3_df.shape

(3, 2900)

In [62]:
uds4_df.shape

(0, 0)

In [63]:
uds3_df['telmod'].value_counts()

telmod
1.0    1
Name: count, dtype: int64

In [64]:
# Identify all float columns
float_columns = uds3_df.select_dtypes(include=['float']).columns

# Convert safely
for col in float_columns:
    # Convert to numeric, coerce errors to NaN
    uds3_df[col] = pd.to_numeric(uds3_df[col], errors='coerce')

    # Replace infinities with NaN
    uds3_df[col] = uds3_df[col].replace([np.inf, -np.inf], np.nan)

    # Check if all remaining values are safe for integer conversion
    if uds3_df[col].dropna().apply(float.is_integer).all():
        uds3_df[col] = uds3_df[col].astype("Int64")  # Convert to nullable integer
    else:
        print(f"Skipping column '{col}' due to non-integer values.")

# Check the updated data types
print(uds3_df.dtypes)


Skipping column 'form_ver_num' due to non-integer values.
Skipping column 'height' due to non-integer values.
Skipping column 'bmi' due to non-integer values.
Skipping column 'inbody_skel_musc_mass' due to non-integer values.
Skipping column 'inbody_body_fat_mass' due to non-integer values.
Skipping column 'inbody_bmi' due to non-integer values.
Skipping column 'inbody_perc_body_fat' due to non-integer values.
Skipping column 'memory' due to non-integer values.
Skipping column 'orient' due to non-integer values.
Skipping column 'commun' due to non-integer values.
Skipping column 'homehobb' due to non-integer values.
Skipping column 'cdrsum' due to non-integer values.
Skipping column 'tot_sum' due to non-integer values.
Skipping column 'cdrglob' due to non-integer values.
Skipping column 'comport' due to non-integer values.
Skipping column 'orient_dist' due to non-integer values.
Skipping column 'judgment_dist' due to non-integer values.
Skipping column 'commun_dist' due to non-integer 

In [65]:
def process_mappings(mapping_type):
    for mapping in mapping_data.get(mapping_type, []):
        uds3_var = mapping["UDS3_variable"].lower()
        uds4_var = mapping["UDS4_variable"].lower()

        uds3_var = data_dict.get(uds3_var, uds3_var)
        structured_mappings = []

        if mapping_type == 'Direct_Mappings':
            response_levels = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_remappings"] if m["mapping_type"] == "Conformity"]
        else:
            response_levels = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Response LEVELS"]
            conformity = [m for m in mapping["crosswalk_mappings"] if m["mapping_type"] == "Conformity"]
            structured_mappings = [m for m in mapping.get("crosswalk_mappings", []) if m["mapping_type"] == "Structured mapping"]

        if mapping_type == 'Structured_Transformations':
            if response_levels and not conformity and uds3_var in uds3_df.columns:
                response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}
                for uds3_value, uds4_value in response_map.items():
                    if "|" not in uds3_value and "grep(" not in uds3_value:
                        uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value

        if structured_mappings:
            print(structured_mappings)
            for struct_map in structured_mappings:
                for mapping_entry in struct_map["mappings"]:
                    uds3_value = mapping_entry["UDS3_value"]
                    uds4_value = mapping_entry["UDS4_value"]

                    # Identify separator: OR ('|') or AND ('&')
                    if "|" in uds3_value:
                        separator = "|"
                        operator = np.logical_or
                    elif "&" in uds3_value:
                        separator = "&"
                        operator = np.logical_and
                    else:
                        separator = None
                        operator = None

                    # Handle multiple UDS3 variables with OR / AND conditions
                    if separator:
                        #print(separator)
                        uds3_vars = [var.strip() for var in uds3_var.split(f" {separator} ")]
                        #print(uds3_vars)
                        uds3_values = [var.strip() for var in uds3_value.split(f" {separator} ")]
                        #print(uds3_values)

                        # Ensure all UDS3 columns exist in the DataFrame
                        if all(col in uds3_df.columns for col in uds3_vars):
                            conditions = [(uds3_df[col].astype(str) == val) for col, val in zip(uds3_vars, uds3_values)]
                            # Apply OR or AND depending on the separator
                            if separator == '|':
                                mask = np.logical_or.reduce(conditions)
                            else:
                                mask = np.logical_and.reduce(conditions)

                            # Only update where conditions match
                            if uds4_var in uds4_df.columns:
                                missing_mask = uds4_df[uds4_var].isna()  # Handle missing (NA) values in uds4_df
                                uds4_df.loc[missing_mask & mask, uds4_var] = uds4_value
                            else:
                                uds4_df.loc[mask, uds4_var] = uds4_value
                    else:
                        # Direct mapping (no separator)
                        if uds3_var in uds3_df.columns:
                            mask = uds3_df[uds3_var].fillna(0).astype(int) == int(uds3_value)
                            uds4_df.loc[mask, uds4_var] = uds4_value

                        # Handle "paste()" format for combining multiple columns into a single string
                        if "paste(" in mapping_entry["UDS4_value"]:
                            components_raw = mapping_entry["UDS4_value"].replace("paste(", "").replace(")", "").split(',')
                            columns = [re.sub(r'__A1$', '', comp.strip()).lower() for comp in components_raw if '__A1' in comp]

                            if all(comp in uds3_df.columns for comp in columns):
                                uds4_df[uds4_var] = (uds3_df[columns[0]].astype(str) + "/" +
                                                     uds3_df[columns[1]].astype(str) + "/" +
                                                     uds3_df[columns[2]].astype(str))

        # Case 1: No Response LEVELS, copy values directly
        if not response_levels:
            if uds3_var in uds3_df.columns:
                uds4_df[uds4_var] = uds3_df[uds3_var]

        # Case 2: Conformity check
        elif conformity:
            uds3_conformity_values = set(item["UDS3_value"] for mapping in conformity for item in mapping["mappings"])
            uds4_conformity_values = set(item["UDS4_value"] for mapping in conformity for item in mapping["mappings"])

            if uds3_conformity_values == uds4_conformity_values:
                if uds3_var in uds3_df.columns:
                    uds4_df[uds4_var] = uds3_df[uds3_var]
            else:
                response_map = {str(item["UDS3_value"]): str(item["UDS4_value"]) for mapping in response_levels for item in mapping["mappings"]}

                if uds3_var in uds3_df.columns:
                    # Apply mappings for exact matches
                    for uds3_value, uds4_value in response_map.items():
                        if "|" not in uds3_value and "grep(" not in uds3_value:
                            uds4_df.loc[uds3_df[uds3_var].astype(str) == uds3_value, uds4_var] = uds4_value

                        # Handle logical OR conditions
                        elif "|" in uds3_value:
                            values = uds3_value.split(" | ")
                            uds4_df.loc[uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value

                        # Handle grep-based text searches
                        for uds3_value, uds4_value in response_map.items():
                            if "grep(" in uds3_value:
                                search_term = re.search(r'grep\("(.*?)",', uds3_value).group(1).lower()
                                uds4_df.loc[uds3_df[uds3_var].astype(str).str.contains(search_term, case=False, na=False), uds4_var] = uds4_value

        # Case 3: Handling secondary and tertiary race variables (race, racesec, raceter)
        if uds3_var.endswith("sec") or uds3_var.endswith("ter"):
            if uds3_var in uds3_df.columns:
                missing_mask = uds4_df[uds4_var] == "NA"  # Identify rows where primary race is NA
                for uds3_value, uds4_value in response_map.items():
                    if "|" not in uds3_value and "grep(" not in uds3_value:
                        uds4_df.loc[missing_mask & (uds3_df[uds3_var].astype(str) == uds3_value), uds4_var] = uds4_value
                    elif "grep(" in uds3_value:
                        search_term = re.search(r'grep\("(.*?)",', uds3_value).group(1).lower()
                        uds4_df.loc[missing_mask & uds3_df[uds3_var].astype(str).str.contains(search_term, case=False, na=False), uds4_var] = uds4_value
                    else:
                        values = uds3_value.split(" | ")
                        uds4_df.loc[missing_mask & uds3_df[uds3_var].astype(str).isin(values), uds4_var] = uds4_value

# Process different mapping types dynamically
mapping_types = ["Direct_Mappings", "Conditional_Consistency", "Structured_Transformations"]  # Add more types as needed
for mapping_type in mapping_types:
    process_mappings(mapping_type)


[{'mapping_type': 'Structured mapping', 'mappings': [{'UDS3_value': '0 & 0 & 0 & 0 & 0', 'UDS4_value': '1', 'reversible': 'Yes', 'note': 'UDS3 T1 specific structured mapping'}]}]
[{'mapping_type': 'Structured mapping', 'mappings': [{'UDS3_value': '1 | 1 | 1 | 1 | 1', 'UDS4_value': '2', 'reversible': 'Yes', 'note': 'UDS3 T1 specific structured mapping'}]}]
[{'mapping_type': 'Structured mapping', 'mappings': [{'UDS3_value': 'strplit(FRMDATEA1, "/")', 'UDS4_value': 'paste(VISITMO__A1, "/", VISITDAY__A1, "/", VISITYR__A1)', 'reversible': 'Yes', 'note': 'Structured mapping presented as R expression'}]}]


In [66]:
uds4_df.modea1.value_counts()

modea1
2    1
Name: count, dtype: int64

In [68]:
# UTF-8-SIG encoding to avoid character corruption
uds4_df.to_csv("uds4_data2.csv", index=False, na_rep="NA", encoding='utf-8-sig')

In [69]:
uds4_df.head(5)

,birthmo,educ,handed,ethhisothx,livsitua,maristat,ptid__a1,refersc,birthsex,sourcenw,...,raceasian,raceblack,racenhpi,raceunkn,racewhite,racemena,residenc,rmreasa1,modea1,frmdatea1
0,2,12,2,Guatemalan,6,2,ADC1105,2,2,1,...,None,None,None,None,None,NaN,3,NaN,NaN,1/14/2025
1,3,12,2,NaN,1,2,ADC1102,3,1,1,...,None,None,None,None,None,NaN,1,NaN,NaN,1/13/2025
2,5,16,2,NaN,2,1,ADC1129,4,2,1,...,None,None,None,None,None,NaN,1,4,2,2/5/2025


In [70]:
uds3_df['hispanic'].value_counts()

hispanic
1    1
0    1
Name: count, dtype: Int64

In [71]:
uds4_df['ethispanic']

0       1
1    None
2     NaN
Name: ethispanic, dtype: object

In [72]:
uds4_df['frmdatea1']

0    1/14/2025
1    1/13/2025
2     2/5/2025
Name: frmdatea1, dtype: object